# Export ArcGIS Metadata as InPort XML

In [25]:
import os, traceback
import arcpy
from arcpy import metadata as md
from lxml import etree
from io import StringIO
import os

# Get the Home Folder and the Default Geodatabase from the CURRENT ArcGIS Pro project
home_folder = arcpy.mp.ArcGISProject("CURRENT").homeFolder
project_gdb = arcpy.mp.ArcGISProject("CURRENT").defaultGeodatabase

export_folder = f"{home_folder}\Metadata Export"
if not arcpy.Exists(export_folder):
    arcpy.management.CreateFolder(home_folder, "Metadata Export")
del export_folder

arcpy.env.overwriteOutput = True
arcpy.env.workspace = project_gdb

# Reference the XSL file to use for the transformation
xsl_file  = rf"{home_folder}\ArcGIS2Inport.xsl"
    
datasets = list()

walk = arcpy.da.Walk(project_gdb)

for dirpath, dirnames, filenames in walk:
    for filename in filenames:
        datasets.append(os.path.join(dirpath, filename))

if len(datasets) == 0:
    print("No datasets found.")
elif len(datasets) > 0:
    print("Working . . .")
    count=0
    for dataset in sorted(datasets):
        count+=1
        print(f"Dataset: {os.path.basename(dataset)} (Count {count} of {len(datasets)})")        
        dataset_md = md.Metadata(dataset)
        #dataset_md.synchronize("ALWAYS")
        #dataset_md.save()
        #print(f"{home_folder}\Metadata Export\{os.path.basename(dataset)}_InPort.xml")

        tree = etree.parse(StringIO(dataset_md.xml), parser=etree.XMLParser(encoding='UTF-8', remove_blank_text=True))
        root = tree.getroot()

        mdParentID = root.find("mdParentID")  # Prod: 65940 Test: 75873, prod dismap 66799
        
        if mdParentID is None:
            _xml = etree.XML(f"<mdParentID>gov.noaa.nmfs.inport:65940</mdParentID>", etree.XMLParser(encoding='UTF-8', remove_blank_text=True))
            root.insert(root_dict["mdParentID"], _xml)
            del _xml
        elif mdParentID is not None:
            mdParentID.text = "gov.noaa.nmfs.inport:65940"
        else:
            none            
        #print(mdParentID.text)
        
        del mdParentID

        etree.indent(tree, "    ")
        dataset_md.xml = etree.tostring(tree, encoding='UTF-8', method='xml', xml_declaration=True, pretty_print=True).decode()
        dataset_md.save()
        dataset_md.synchronize("ALWAYS")
        dataset_md.save()
        
        try:
            dataset_md.exportMetadata(f"{home_folder}\Metadata Export\{os.path.basename(dataset)}_InPort.xml", "CUSTOM", "EXACT_COPY", xsl_file)
        except arcpy.ExecuteError:
            print(arcpy.GetMessages())
        except:
            traceback.print_exc()
            arcpy.GetMessages()
        del dataset_md
        del dataset
    print("Work Complete")
else:
    pass
del datasets
del etree, md, os, StringIO
    

Working . . .
Dataset: AbaloneBlack_20250521 (Count 1 of 150)
Dataset: AbaloneWhite_20250529 (Count 2 of 150)
Dataset: All_Species_Ranges_20250710 (Count 3 of 150)
Dataset: AngelsharkArgentine_20250609 (Count 4 of 150)
Dataset: AngelsharkCommon_20250609 (Count 5 of 150)
Dataset: AngelsharkSawback_20250609 (Count 6 of 150)
Dataset: AngelsharkSmoothback_20250609 (Count 7 of 150)
Dataset: AngelsharkSpiny_20250609 (Count 8 of 150)
Dataset: Bocaccio_PugetSoundGeorgiaBasinDPS_20250519 (Count 9 of 150)
Dataset: CardinalfishBanggai_20250626 (Count 10 of 150)
Dataset: CoelacanthAfricanTanzanianDPS_20250528 (Count 11 of 150)
Dataset: ConchQueen_20250603 (Count 12 of 150)
Dataset: CoralAcroporaGlobiceps_20250521 (Count 13 of 150)
Dataset: CoralAcroporaJacquelineae_20250625 (Count 14 of 150)
Dataset: CoralAcroporaLokani_20250625 (Count 15 of 150)
Dataset: CoralAcroporaPharaonis_20250625 (Count 16 of 150)
Dataset: CoralAcroporaRetusa_20250521 (Count 17 of 150)
Dataset: CoralAcroporaRudis_20250625 (

# User Credentials

In [18]:
# Hard coded
credentials={"username" : "firstname.lastname@noaa.gov", "password" : "NOAA_LDAP_PASSWORD"}
# OR
# Project (Home) Folder from currently opened ArcGIS Pro
home_folder = arcpy.mp.ArcGISProject("CURRENT").homeFolder
# Load user_credentials.py to get user credentials
# def main():
#     credentials={"username" : "firstname.lastname@noaa.gov", "password" : "NOAA_LDAP_PASSWORD"}
#     return credentials
# if __name__ == '__main__':
#     main()
# Imports
import os
# Change Directory
os.chdir(home_folder)
import user_credentials
credentials = user_credentials.main()

## Contacts API Specs ###
### Endpoints ###
#### /inport/api/find-contact ####
#### (POST) ####

In [31]:
import requests, json, copy
inport_url = "https://www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response    
    
    session_id_dict["contactType"]  = "Person"
    session_id_dict["emailAddress"] = "jennifer.schultz@noaa.gov"
    
    _url = "https://www.fisheries.noaa.gov/inport/api/find-contact"
    response = requests.get(_url, params = session_id_dict)        
    #print(response.json())
    _dict = copy.deepcopy(response.json())
    
    response.close()
    del response, _url
    
    for key in _dict:
        print(key)
        if isinstance(_dict[key], dict):
            for k in _dict[key]:
                print(f"\t{k}: {_dict[key][k]}")
        elif isinstance(_dict[key], list):
            for i in range(0, len(_dict[key])):
                for k in _dict[key][i]:
                    print(f"\t{k}: {_dict[key][i][k]}")


elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT 
# Authentication successful!
# {'sessionId': '75c3628752d91a126fcbded22998f3e'}
# results
# 	lastName: Kennedy
# 	firstName: John
# 	middleName: F
# 	phone: 301-427-8149
# 	emailAddress: john.f.kennedy@noaa.gov
# 	address: 1315 East-West Highway
# 	city: Silver Spring
# 	stateProvince: MD
# 	postalCode: 20910
# 	country: USA
# 	contactType: Person
# 	rorMaxLength: 9
# 	orcIdMaxLength: 19

Authentication successful!
{'sessionId': '7340dbd8e69622674f4bbcce'}
results
	lastName: Schultz
	firstName: Jennifer
	emailAddress: jennifer.schultz@noaa.gov
	contactType: Person
	orcIdMaxLength: 19
	rorMaxLength: 9


## Search API Specs ###
### Endpoints ###
#### /inport/api/search ####
#### (GET) ####

In [ ]:
import requests, json, copy
inport_url = "https://www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    #print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response       
    
    session_id_dict["keywords"]    = "environment"
    session_id_dict["org"]         = "OST"
    session_id_dict["catItemType"] = "DS,ENT,PRJ"
    print(session_id_dict)

    _url = "https://www.fisheries.noaa.gov/inport/api/search"
    response = requests.get(_url, params = session_id_dict)        
    #print(response.json())
    _dict = copy.deepcopy(response.json())
    response.close()
    del response, _url
    
    for key in _dict:
        print(key)
        if isinstance(_dict[key], dict):
            for k in _dict[key]:
                print(f"\t{k}: {_dict[key][k]}")
        elif isinstance(_dict[key], list):
            for i in range(0, len(_dict[key])):
                for k in _dict[key][i]:
                    print(f"\t{k}: {_dict[key][i][k]}")

elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT 
# Authentication successful!
# {'sessionId': '13a880c7ba85d337a572d92f', 'keywords': 'environment', 'org': 'OST', 
# 'catItemType': 'DS,ENT,PRJ'}
# results
# 	catId: 20784
# 	catItemType: Project
# 	org: OST
# 	title: Large Marine Ecosystems (LME)
# 	abstractValue: Large Marine Ecosystems (LME) are ocean regions of 200,000 km2 or greater 
#   that are defined by ecological criteria, including bathymetry, hydrography, marine 
#   productivity, and trophically linked populations.  Since 1984, NOAA?s LME Program has 
#   developed ecosystem management tools, initiated projects that have been funded by partner 
#   organizations, and provided training for developing country part...
	

### Load InPort XML ###
#### /inport/api/load-inport-xml ####
#### (POST) ####

In [23]:
# Imports
import requests, json, copy, os
import os
# Change Directory
os.chdir(home_folder)
import user_credentials
credentials = user_credentials.main()

# InPort URL
#inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_url = "https://www.fisheries.noaa.gov/inport"

# Use requests get method to log into InPort
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))

# Test if successful i.e. status code 200
if inport_response.status_code == 200:
    print("Authentication successful!")
    
    #xml = rf"{home_folder}\Metadata Export\PolygonExample_InPort.xml"
    #xml = rf"{home_folder}\Metadata Export\AbaloneBlack_20250521_InPort.xml"

    #xmls = [rf"{home_folder}\Metadata Export\AbaloneWhite_20250529_InPort.xml", rf"{home_folder}\Metadata Export\All_Species_Ranges_20250710_InPort.xml"]
    xmls = [rf"{home_folder}\Metadata Export\{xml}" for xml in os.listdir(rf"{home_folder}\Metadata Export") if xml.endswith("_InPort.xml")]
        
    for xml in xmls:
        print(f"XML File: {os.path.basename(xml)}")

        # InPort URL to get a session id
        #_url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
        _url = "https://www.fisheries.noaa.gov/inport/api/get-session-id"
        
        # Use requests post method to get a session id for credentials pasted to InPort
        session_id_response = requests.post(_url, json = credentials)
        
        # Make a deep copy of the response, using the method json() to return a Python dictionary
        session_id_dict = copy.deepcopy(session_id_response.json())
        
        # Close the response object and then delete variable
        session_id_response.close()
        del _url, session_id_response
        
        # Opening a file for reading
        with open(xml, "r") as file:
            content = file.read()
            
        # Insert XML content from file into the dictionary, using 'xml' as the key,     
        session_id_dict["xml"] = content
     
        #_url = "https://test-www.fisheries.noaa.gov/inport/api/load-inport-xml"
        _url = "https://www.fisheries.noaa.gov/inport/api/load-inport-xml"
        # headers = {"Content-Type": "application/json"}     
        # response = requests.post(url, json=payload, headers=headers)
        # response = requests.post(_url, json=session_id_dict, headers=headers)
        # Use request post method to load content to InPort
        response = requests.post(_url, json=session_id_dict)
        
        if response.status_code == 200:
            print(response)
            _dict = copy.deepcopy(response.json())
            response.close()
        
            for key in _dict:
                print(key)
                if isinstance(_dict[key], dict):
                    for k in _dict[key]:
                        print(f"\t{k}: {_dict[key][k]}")
                elif isinstance(_dict[key], list):
                    for i in range(0, len(_dict[key])):
                        for k in _dict[key][i]:
                            print(f"\t{k}: {_dict[key][i][k]}")
                        print()
  
        elif response.status_code == 400:
            print(session_id_dict)    
            print(response.content)        
        
        else:
            print(f"Something is wrong with load-inport-xml!!\n\tStatus Code: {response.status_code}\n\tContent: {response.content}")
    
        del response, _url

elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close() 
del inport_response, inport_url
# RESULT ###--->>> Authentication successful!

Authentication successful!
XML File: WhaleBeluga_CookInletDPS_20250624_InPort.xml
<Response [200]>
catId
errors
warnings
childItems
	catId: 76655
	errors: []
	warnings: []
	childItems: []

XML File: WhaleBlue_20250627_InPort.xml
<Response [200]>
catId
errors
warnings
childItems
	catId: 76657
	errors: []
	warnings: []
	childItems: []

XML File: WhaleBowhead_20250627_InPort.xml
<Response [200]>
catId
errors
warnings
childItems
	catId: 76659
	errors: []
	warnings: []
	childItems: []

XML File: WhaleFalseKiller_MainHawaiianIslandsInsularDPS_20250624_InPort.xml
<Response [200]>
catId
errors
warnings
childItems
	catId: 76661
	errors: []
	warnings: []
	childItems: []

XML File: WhaleFinback_20250627_InPort.xml
<Response [200]>
catId
errors
warnings
childItems
	catId: 76663
	errors: []
	warnings: []
	childItems: []

XML File: WhaleGray_WesternNorthPacificDPS_20250627_InPort.xml
<Response [200]>
catId
errors
warnings
childItems
	catId: 76665
	errors: []
	warnings: []
	childItems: []

XML File: 

In [22]:
import os
home_folder = arcpy.mp.ArcGISProject("CURRENT").homeFolder
print([xml for xml in os.listdir(rf"{home_folder}\Metadata Export") if xml.endswith("_InPort.xml") and xml.startswith("Abalone")])

[]
